In [1]:
import numpy as np

In [2]:
# Define the grid for gridworld (including the out of bounds states)
grid = np.array([
    [0, 10, 0, 5, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0]
])
grid

array([[ 0, 10,  0,  5,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0]])

In [3]:
# Probs for each state
grid_probs = np.array([
    [2, 3, 3, 3, 2],
    [3, 4, 4, 4, 3],
    [3, 4, 4, 4, 3],
    [3, 4, 4, 4, 3],
    [2, 3, 3, 3, 2]
]).astype(float)
grid_probs /= grid_probs.sum()
grid_probs

array([[0.025 , 0.0375, 0.0375, 0.0375, 0.025 ],
       [0.0375, 0.05  , 0.05  , 0.05  , 0.0375],
       [0.0375, 0.05  , 0.05  , 0.05  , 0.0375],
       [0.0375, 0.05  , 0.05  , 0.05  , 0.0375],
       [0.025 , 0.0375, 0.0375, 0.0375, 0.025 ]])

In [4]:
def policy(state):
    # Policy returns equal probabilities for each direction
    return np.array([0.25, 0.25, 0.25, 0.25])

In [5]:
def reward(state):
    # Reward is -1 if out of bounds, else 0
    i, j = state
    if i < 0 or j < 0 or i >= grid.shape[0] or j >= grid.shape[1]:
        return -1
    return 0

In [6]:
def state_transition(state, action):
    # If the state is (0, 1), next state is (4, 1) and reward is 10
    if state[0] == 0 and state[1] == 1:
        return (4, 1), 10
    # If the state is (0, 3), next state is (2, 3) and reward is 5
    if state[0] == 0 and state[1] == 3:
        return (2, 3), 5
    
    # North is 0, East is 1, South is 2, West is 3
    if action == 0:
        state_ = (state[0]-1, state[1])
    elif action == 1:
        state_ = (state[0], state[1]+1)
    elif action == 2:
        state_ = (state[0]+1, state[1])
    elif action == 3:
        state_ = (state[0], state[1]-1)
        
    # Get reward for the next state (-1 or 0)
    r = reward(state_)
        
    # If next state is greater than the bounds, move in bounds
    state_ = (min(grid.shape[0]-1, max(0, state_[0])), min(grid.shape[1]-1, max(0, state_[1])))
    
    return state_, r

In [7]:
# Discount factor
discount = 0.9

In [8]:
# state-values initialized to zeros
values = np.zeros_like(grid, dtype=float)

In [9]:
# Recursive depth
depth = 9

# Function used to calculate the state_value of a given state
def calculate_state_value(state, d):
    if d == depth:
        return 0
    
    # Sample policy probabilities
    probs = policy(state)
    
    total_value = 0
    
    # Iterate over all possible actions
    for a in range(0, len(probs)):
        # Policy probability pi(a | s)
        p = probs[a]
        
        # With probability 1, we go to the next state
        state_, r = state_transition(state, a)
        
        # Calculate value of future state
        v = calculate_state_value(state_, d+1)
        
        # Sum of future states is just the next state as
        # we deterministically move to the next state
        # p(s', r | s, a) = 1
        sum_ = 1 * (r + discount * v)
        
        # Weigh by the policy probability
        value = p * sum_
        
        # Add to total value
        total_value += value
        
    return total_value
        
    

# Iterate for all i,j on the grid
for i in range(0, grid.shape[0]):
    for j in range(0, grid.shape[1]):
        # State is (i, j)
        values[i, j] = calculate_state_value((i, j), 0)

In [10]:
values.round(2)

array([[ 3.33,  8.93,  4.44,  5.26,  1.35],
       [ 1.47,  2.97,  2.21,  1.82,  0.42],
       [-0.  ,  0.72,  0.63,  0.33, -0.45],
       [-0.93, -0.4 , -0.28, -0.52, -1.1 ],
       [-1.74, -1.21, -1.09, -1.26, -1.8 ]])